In [6]:
import pandas as pd
import numpy as np

# ============================================================
# Files (must be in the same directory as this script)
# ============================================================
AMCE_PATH = "amce_all_groups.csv"
DATA_PATH = "data.xlsx"   # <-- your data is XLSX

# ============================================================
# Settings
# ============================================================
ID_COL = "obs_id"  # numeric unique IDs: 1..N

# Map your dataset column names -> AMCE "Attribute" names
COL_TO_ATTRIBUTE = {
    "ArrivalYear": "Arrival",
    "City": "City",
    "AdultEdu": "Education",
    "HouseholdComp": "Demographics",
    "Disability": "Disability",
    "MedicalCond": "Medical",
    "HH_Assets": "Assets",
    "DebtSituation": "Debt",
    "Coping": "Coping",
    "AssistanceHistory": "Previous Assistance",
}

GROUPS = ["Beneficiary", "Model", "Field", "Central"]

# ============================================================
# Helpers
# ============================================================
def normalize_level(x) -> str:
    """
    Convert values to a canonical string that matches AMCE 'Level' better.
    - trims whitespace
    - converts nan/None to ""
    - converts floats that are actually ints (2016.0 -> "2016")
    """
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if s.lower() in {"nan", "none", ""}:
        return ""
    # Fix common numeric formatting: "2016.0" -> "2016"
    try:
        f = float(s)
        if f.is_integer():
            return str(int(f))
        return s
    except Exception:
        return s

# ============================================================
# Load AMCEs and build lookup per group
# ============================================================
amce = pd.read_csv(AMCE_PATH)

needed_cols = {"Group", "Attribute", "Level", "Estimate"}
missing_amce_cols = sorted(list(needed_cols - set(amce.columns)))
if missing_amce_cols:
    raise ValueError(
        f"amce_all_groups.csv is missing columns: {missing_amce_cols}. "
        f"Found columns: {list(amce.columns)}"
    )

amce = amce[["Group", "Attribute", "Level", "Estimate"]].copy()
amce["Group"] = amce["Group"].astype(str).str.strip()
amce["Attribute"] = amce["Attribute"].astype(str).str.strip()
amce["Level"] = amce["Level"].apply(normalize_level)

group_lookup = {}
for g, gdf in amce.groupby("Group", sort=False):
    group_lookup[g] = {
        (row.Attribute, row.Level): float(row.Estimate)
        for row in gdf.itertuples(index=False)
    }

missing_groups = [g for g in GROUPS if g not in group_lookup]
if missing_groups:
    raise ValueError(
        f"These groups are missing in amce_all_groups.csv: {missing_groups}. "
        f"Found groups: {sorted(group_lookup.keys())}"
    )

# ============================================================
# Load XLSX data and create numeric unique IDs
# ============================================================
# If your data is not in the first sheet, change sheet_name (e.g., sheet_name="Sheet2")
X = pd.read_excel(DATA_PATH, sheet_name=0)

required_cols = list(COL_TO_ATTRIBUTE.keys())
missing_data_cols = [c for c in required_cols if c not in X.columns]
if missing_data_cols:
    raise ValueError(
        f"data.xlsx is missing columns: {missing_data_cols}\n"
        f"Found columns: {list(X.columns)}"
    )

X = X.copy()
X[ID_COL] = np.arange(1, len(X) + 1, dtype=int)

# Normalize data levels to match AMCE levels
X_norm = X.copy()
for c in required_cols:
    X_norm[c] = X_norm[c].apply(normalize_level)

# ============================================================
# Scoring
# ============================================================
def score_observations_for_group(Xn: pd.DataFrame, lookup: dict) -> pd.Series:
    """
    Score each row by summing AMCE estimates for realized (Attribute, Level).
    If (Attribute, Level) isn't in AMCE => treated as control => +0.
    """
    scores = np.zeros(len(Xn), dtype=float)

    for col, attr in COL_TO_ATTRIBUTE.items():
        levels = Xn[col].to_numpy(dtype=str)
        add = np.fromiter(
            (lookup.get((attr, lvl), 0.0) for lvl in levels),
            dtype=float,
            count=len(levels),
        )
        scores += add

    return pd.Series(scores, index=Xn.index, name="score")

# ============================================================
# Compute scores + rankings for each group
# ============================================================
results = {}

for g in GROUPS:
    lookup = group_lookup[g]
    scores = score_observations_for_group(X_norm, lookup)

    tmp = pd.DataFrame({
        ID_COL: X_norm[ID_COL].values,
        "score": scores.values
    })

    ranked = tmp.sort_values(["score", ID_COL], ascending=[False, True], kind="mergesort").reset_index(drop=True)
    ranked["rank"] = np.arange(1, len(ranked) + 1, dtype=int)

    results[g] = {
        "scores_table": tmp,                         # obs_id + score (original order)
        "ranking_vector": ranked[ID_COL].to_list(),  # ordered vector of obs_id
        "ranking_table": ranked,                     # obs_id + score + rank (sorted)
    }

# ============================================================
# Outputs
# ============================================================
# 1) One CSV per group: obs_id, score, rank (sorted)
for g in GROUPS:
    results[g]["ranking_table"].to_csv(f"ranking_{g.lower()}.csv", index=False)

# 2) One wide CSV: obs_id + all group scores (unsorted)
scores_wide = pd.DataFrame({ID_COL: X_norm[ID_COL].values})
for g in GROUPS:
    scores_wide[f"score_{g.lower()}"] = results[g]["scores_table"]["score"].values
scores_wide.to_csv("scores_all_groups.csv", index=False)

# 3) Convenience: print top-10 IDs per group
for g in GROUPS:
    print(f"Top 10 obs_id ({g}): {results[g]['ranking_vector'][:10]}")

print("\nWrote files:")
for g in GROUPS:
    print(f" - ranking_{g.lower()}.csv")
print(" - scores_all_groups.csv")


PermissionError: [Errno 13] Permission denied: 'data.xlsx'